In [1]:
# import libraries 
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2023-02-27 17:45:15--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   101MB/s    in 0.2s    

2023-02-27 17:45:15 (101 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [10]:
#Data cleaning according to requirement
df_new_ratings_1 = df_ratings.merge(df_ratings.user.value_counts()[df_ratings.user.value_counts()>=200], left_on = 'user', right_index=True).reset_index()[['user','isbn','rating']]
df_new_ratings_2 = df_new_ratings_1.merge(df_ratings.isbn.value_counts()[df_ratings.isbn.value_counts()>=100], left_on = 'isbn', right_index=True).reset_index()[['user','isbn','rating']]

#merge to get new df to fit into model
df_new = df_books.reset_index()
df_new_ratings = df_new_ratings_2.merge(df_new[['index','isbn','title']],left_on = 'isbn',right_on = 'isbn')[['index','user','isbn','rating','title']].drop_duplicates(subset = ['title','user'])

In [11]:
#Fit the model using k-Nearest Neighbourhood with k=5
X = csr_matrix((np.array(df_new_ratings.rating),(np.array(df_new_ratings['index']), np.array(df_new_ratings.user))))
neigh = NearestNeighbors(n_neighbors=5,metric = 'cosine')
neigh.fit(X)

NearestNeighbors(metric='cosine')

In [8]:
# function to return recommended books
def get_recommends(book = ""):
  recommended_books = []
  recommended_books.append(book)
  ret_list = []
  book_isbn = df_books.loc[df_books['title'] == book].isbn
  book_ratings = df_new_ratings.loc[(df_new_ratings['isbn'] == list(book_isbn)[0])]
  ret = neigh.kneighbors(csr_matrix((np.array(book_ratings.rating),([0]*len(np.array(book_ratings['index'])), np.array(book_ratings.user))),shape=(1,X.shape[1])))
  ret_list = [[list(df_books.iloc[ret[1][0]]['title'])[i], ret[0][0][i]]  for i in range(1,5)][::-1]
  recommended_books.append(ret_list)
  return recommended_books

In [9]:
#Unit test
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
